In [1]:
import torch.multiprocessing
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms

from corrosion_dataset import CorrosionSegmentationDataset
from unet import UNet

torch.multiprocessing.set_start_method("spawn", force=True)  # ✅ Must be the first line

# ✅ Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# ✅ Reduce num_workers to prevent crashing
BATCH_SIZE = 16
NUM_WORKERS = 2  # ✅ Reduce for stability (or set to 0 if still crashing)
GRAD_ACCUM_STEPS = 2  # ✅ Gradient accumulation

# Set dataset paths
image_dir = "processed_dataset"
mask_dir = "processed_dataset_masks"

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),  # Normalize grayscale images
])

# ✅ Load dataset inside `if __name__ == "__main__"`
dataset = CorrosionSegmentationDataset(image_dir, mask_dir, transform=transform)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

# ✅ Initialize model inside `if __name__ == "__main__"`
model = UNet(num_classes=2).to(device)

C:\Users\Kyojuro\MLProjects\SEMClassify\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Kyojuro\MLProjects\SEMClassify\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=FCN_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
0.6%

Downloading: "https://download.pytorch.org/models/fcn_resnet50_coco-1167a1af.pth" to C:\Users\Kyojuro/.cache\torch\hub\checkpoints\fcn_resnet50_coco-1167a1af.pth


100.0%


In [3]:
# ✅ Define loss function & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-4)
scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision (FP16)

# ✅ Training loop
for epoch in range(20):
    model.train()
    total_loss = 0
    correct_pixels = 0
    total_pixels = 0
    optimizer.zero_grad()

    for i, (images, masks) in enumerate(train_loader):
        images, masks = images.to(device), masks.to(device)

        with torch.cuda.amp.autocast():  # ✅ Enable FP16
            outputs = model(images)
            loss = criterion(outputs, masks.long()) / GRAD_ACCUM_STEPS  # ✅ Gradient accumulation

        scaler.scale(loss).backward()

        if (i + 1) % GRAD_ACCUM_STEPS == 0:  # ✅ Update weights after accumulation steps
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct_pixels += (preds == masks).sum().item()
        total_pixels += masks.numel()

    pixel_accuracy = 100 * correct_pixels / total_pixels
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Pixel Accuracy: {pixel_accuracy:.2f}%")

torch.save(model.state_dict(), "unet_corrosion.pth")
print("✅ Training complete. Model saved.")

C:\Users\Kyojuro\AppData\Local\Temp\ipykernel_6076\2934163907.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # ✅ Mixed Precision (FP16)
C:\Users\Kyojuro\AppData\Local\Temp\ipykernel_6076\2934163907.py:17: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # ✅ Enable FP16


Epoch 1, Loss: 2.1189, Pixel Accuracy: 76.20%
Epoch 2, Loss: 1.4659, Pixel Accuracy: 85.85%
Epoch 3, Loss: 1.2100, Pixel Accuracy: 87.64%
Epoch 4, Loss: 1.1649, Pixel Accuracy: 88.99%
Epoch 5, Loss: 1.0577, Pixel Accuracy: 90.12%
Epoch 6, Loss: 0.9846, Pixel Accuracy: 90.77%
Epoch 7, Loss: 0.8928, Pixel Accuracy: 91.55%
Epoch 8, Loss: 0.8946, Pixel Accuracy: 91.54%
Epoch 9, Loss: 0.8804, Pixel Accuracy: 91.93%
Epoch 10, Loss: 0.7833, Pixel Accuracy: 92.40%
Epoch 11, Loss: 0.8319, Pixel Accuracy: 92.17%
Epoch 12, Loss: 0.8142, Pixel Accuracy: 92.51%
Epoch 13, Loss: 0.7544, Pixel Accuracy: 92.62%
Epoch 14, Loss: 0.7723, Pixel Accuracy: 92.71%
Epoch 15, Loss: 0.8460, Pixel Accuracy: 92.66%
Epoch 16, Loss: 0.7270, Pixel Accuracy: 93.35%
Epoch 17, Loss: 0.6784, Pixel Accuracy: 93.42%
Epoch 18, Loss: 0.6959, Pixel Accuracy: 93.55%
Epoch 19, Loss: 0.6795, Pixel Accuracy: 93.77%
Epoch 20, Loss: 0.6594, Pixel Accuracy: 93.87%
✅ Training complete. Model saved.
